In [0]:
!pip install autocorrect

    100% |████████████████████████████████| 3.6MB 11.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/bf/b8/ae/704d5643f1d0637c5b87d9feccf2ee923c492b703bb0bfbb19
Successfully built autocorrect


In [0]:
!pip install nltk

In [0]:
pwd

In [0]:

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd drive/My Drive/

/content/drive/My Drive


In [0]:
cd quora-question-duplicates/

/content/drive/My Drive/quora-question-duplicates


In [0]:
import pandas as pd
df = pd.read_csv('data/questions.csv')

In [0]:
from sklearn.model_selection import train_test_split

trainDf,testDf = train_test_split(df, test_size=0.2,stratify=df['is_duplicate'].values)
trainDf.to_csv('data/train.csv',index=False)
testDf.to_csv('data/test.csv',index=False)

In [0]:
ls

data/  demo/  LICENSE  models/  README.md  requirement.txt


In [0]:
cd models

/content/drive/My Drive/quora-question-duplicates/models


In [0]:
ls

data_processing.py    Model.03-0.7335.hdf5  Model.07-0.7930.hdf5
embeddings.py         Model.04-0.7507.hdf5  tf_idf_scores.py
Model.01-0.6896.hdf5  Model.05-0.7753.hdf5  topic_modeling.py
Model.02-0.7158.hdf5  Model.06-0.7849.hdf5  utils.py


In [0]:
!python data_processing.py

completed 10000
completed 20000
completed 30000
completed 40000
completed 50000
completed 60000
completed 70000
completed 80000
Finished Question1
completed 90000
completed 100000
completed 110000
completed 120000
completed 130000
completed 140000
completed 150000
completed 160000
Finished Question2
Finished processing file ../data/test.csv
completed 170000
completed 180000
completed 190000
completed 200000
completed 210000
completed 220000
completed 230000
completed 240000
completed 250000
completed 260000
completed 270000
completed 280000
completed 290000
completed 300000
completed 310000
completed 320000
completed 330000
completed 340000
completed 350000
completed 360000
completed 370000
completed 380000
completed 390000
completed 400000
completed 410000
completed 420000
completed 430000
completed 440000
completed 450000
completed 460000
completed 470000
completed 480000
Finished Question1
completed 490000
completed 500000
completed 510000
completed 520000
completed 530000
completed

In [0]:
import os
import pandas as pd
def get_processed_df(data_dir='../data',file_suffix='processed.csv'):
    test_fp = os.path.join(data_dir,'test_'+file_suffix)
    train_fp = os.path.join(data_dir,'train_'+file_suffix)
    train_df = pd.read_csv(train_fp)
    # Drop NA values
    train_df = train_df.dropna()
    test_df = pd.read_csv(test_fp)
    test_df = test_df.dropna()
    return train_df,test_df
  
train_df,test_df = get_processed_df()

In [0]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(
    char_level=True,
    filters=None,
    lower=False,
    num_words=7000
)

uniq_questions = pd.concat((train_df['question1'],train_df['question2'])).unique()




Using TensorFlow backend.


In [0]:
tokenizer.fit_on_texts(uniq_questions)
word_index = tokenizer.word_index #unique words in corpus (training and test sets)

print("Chars in index: %d" % len(word_index))
#sequences = tokenizer.texts_to_sequences(x_train)

Chars in index: 39


In [0]:
q1_char_seq_train = tokenizer.texts_to_sequences(train_df['question1'].values)
q2_char_seq_train = tokenizer.texts_to_sequences(train_df['question2'].values)
q1_char_seq_test = tokenizer.texts_to_sequences(test_df['question1'].values)
q2_char_seq_test = tokenizer.texts_to_sequences(test_df['question2'].values)

In [0]:
max_len=0
for seq in q1_char_seq_test:
  if len(seq) > max_len:
    max_len = len(seq)
print(max_len)

418


In [0]:

MAX_LEN = 1150
from keras.preprocessing.sequence import pad_sequences
q1_data = pad_sequences(q1_char_seq_train, maxlen=MAX_LEN)
q2_data = pad_sequences(q2_char_seq_train, maxlen=MAX_LEN)
test1_data=pad_sequences(q1_char_seq_test, maxlen=MAX_LEN)
test2_data=pad_sequences(q2_char_seq_test, maxlen=MAX_LEN)

labels = train_df['is_duplicate'].values

In [0]:

from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation,GlobalAveragePooling1D,Lambda,Bidirectional
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam, RMSprop
from keras import backend as K

In [0]:
def vec_distance(vects):
    x, y = vects
    return K.sum(K.square(x - y), axis=1, keepdims=True)

In [0]:
def vec_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

In [0]:
from keras.layers.embeddings import Embedding

nb_words=40
max_sentence_len=MAX_LEN
embedding_layer = Embedding(nb_words,300,
        input_length=max_sentence_len)

In [0]:

lstm_layer =LSTM(128)

sequence_1_input = Input(shape=(max_sentence_len,), dtype='int32')
embedded_sequences_1 = embedding_layer(sequence_1_input)
x1 = lstm_layer(embedded_sequences_1)

sequence_2_input = Input(shape=(max_sentence_len,), dtype='int32')
embedded_sequences_2 = embedding_layer(sequence_2_input)
y1 = lstm_layer(embedded_sequences_2)

distance=Lambda(vec_distance, output_shape=vec_output_shape)([x1, y1])
dense1=Dense(16, activation='sigmoid')(distance)
dense1 = Dropout(0.3)(dense1)

bn2 = BatchNormalization()(dense1)
prediction=Dense(1, activation='sigmoid')(bn2)

model = Model(input=[sequence_1_input, sequence_2_input], output=prediction)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


In [0]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1150)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1150)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1150, 300)    12000       input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 128)          219648      embedding_1[0][0]                
          

In [0]:
from keras.callbacks import ModelCheckpoint
model.compile(loss='binary_crossentropy',
        optimizer='adam',
        metrics=['acc'])

filepath='Model.{epoch:02d}-{val_acc:.4f}.hdf5'
checkpointer = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False, mode='auto',period=1)

#model.fit([q1_data, q2_data], labels, validation_data=([test1_data, test2_data], test_df['is_duplicate'].values), verbose=1, 
 #         epochs=10, batch_size=256, shuffle=True,callbacks=[checkpointer])

In [0]:
# Load saved weights
model.load_weights('Model.05-0.8236.hdf5')

model.fit([q1_data, q2_data], labels, validation_data=([test1_data, test2_data], test_df['is_duplicate'].values), verbose=1, 
          epochs=10, batch_size=256, shuffle=True,callbacks=[checkpointer])

Instructions for updating:
Use tf.cast instead.
Train on 323465 samples, validate on 80869 samples
Epoch 1/10
323465/323465 [==============================] - 5504s 17ms/step - loss: 0.3194 - acc: 0.8615 - val_loss: 0.4367 - val_acc: 0.8047

Epoch 00001: saving model to Model.01-0.8047.hdf5
Epoch 2/10
323465/323465 [==============================] - 5452s 17ms/step - loss: 0.3336 - acc: 0.8555 - val_loss: 0.4202 - val_acc: 0.8197

Epoch 00002: saving model to Model.02-0.8197.hdf5
Epoch 3/10
323465/323465 [==============================] - 5459s 17ms/step - loss: 0.2995 - acc: 0.8720 - val_loss: 0.4152 - val_acc: 0.8179

Epoch 00003: saving model to Model.03-0.8179.hdf5
Epoch 4/10
323465/323465 [==============================] - 5443s 17ms/step - loss: 0.2864 - acc: 0.8778 - val_loss: 0.4247 - val_acc: 0.8239

Epoch 00004: saving model to Model.04-0.8239.hdf5
Epoch 5/10
323465/323465 [==============================] - 5483s 17ms/step - loss: 0.2807 - acc: 0.8808 - val_loss: 0.4226 - val